# 盘中动态:20241118

In [1]:
import nbimporter
from 盘前作业20241118 import plotStock
import datetime
import os
import pandas as pd
import akshare as ak

## 1. 今日板块热点


![今日涨幅国泰君安APP](gtja.png)


### 1.1 集合竞价概念板块排名

In [2]:
#记录排名
# basename: 板块类型（概念板块、行业板块）
# start_time, end_time: 排名时间段
# sector: 板块名称
def recordRank(basename,start_time,end_time):
    now = datetime.datetime.now()
    tStr = end_time.strftime("%Y%m%d%H%M")
    csvfile=f'{basename}{tStr}.txt'
    if now.time() < end_time and now.time() >= start_time:
        if basename=='conceptRank':
            rank=ak.stock_board_concept_name_em()
        elif basename=='industryRank':
            rank=ak.stock_board_industry_name_em()
        rank.to_csv(csvfile)
    else:
        if os.path.exists(csvfile):
            rank=pd.read_csv(csvfile)
        else:
            print('非指定时间段，没有数据源.')
            rank=pd.DataFrame()
    if not rank.empty:
        mvtotal=rank['总市值'].sum()
        rank['市值比']=rank['总市值']/mvtotal
        rank['上涨家数%']=rank['上涨家数']/(rank['上涨家数']+rank['下跌家数'])
        columns=['排名','板块名称','板块代码','涨跌幅','市值比','换手率','上涨家数%','领涨股票','领涨股票-涨跌幅']
        rank=rank[columns]
    return rank

#排名靠前的板块的股票明细排名
# sectorRankDf: 板块排名DataFrame(recordRank生成)
# sectorNum,stockNum: 返回板块数、股票数
def top5sectorStocks(basename,start_time,end_time,sectorRankDf,sectorNum=5,stockNum=5):
    top5={}
    now = datetime.datetime.now()
    tStr = start_time.strftime("%Y%m%d%H%M")
    if sectorRankDf.empty:
        print('非指定时间段，没有数据源。')
    else:
        print('排名前5领涨板块成分股列表。')
        for sym in sectorRankDf['板块名称'].to_list()[:5]:
            csvfile=f'{basename}_{sym}{tStr}.txt'
            if now.time() < end_time and now.time() >= start_time:
                if basename=='conceptRank':
                    top5[sym]=ak.stock_board_concept_cons_em(sym)
                else:
                    top5[sym]=ak.stock_board_industry_cons_em(sym)
                top5[sym].to_csv(csvfile)
            else:
                if os.path.exists(csvfile):
                    top5[sym]=pd.read_csv(csvfile)
                else:
                    print('No record stored and not available from source.')
                    top5[sym]=pd.DataFrame()

            if not top5[sym].empty:
                atotal=top5[sym]['成交额'].sum()
                top5[sym]['成交额比']=top5[sym]['成交额']/atotal
                columns=['代码','名称','涨跌幅','成交额','换手率','市盈率-动态','市净率']
                top5[sym]=top5[sym][columns][:stockNum]
                print(f'板块：{sym}:\n{top5[sym]}\n\n')
    return top5   

In [3]:
#当天集合竞价阶段概念板块排名
basename='conceptRank'
start_time = datetime.time(9, 15)
end_time = datetime.time(9, 31)
rank0=recordRank(basename,start_time,end_time)
rank0.head(20)

,排名,板块名称,板块代码,涨跌幅,市值比,换手率,上涨家数%,领涨股票,领涨股票-涨跌幅
0,1,昨日涨停_含一字,BK1050,11.21,0.000399,0.0,1.000000,立方控股,30.00
1,2,昨日涨停,BK0815,11.12,0.000375,0.0,1.000000,立方控股,30.00
2,3,昨日连板_含一字,BK1051,10.37,0.000067,0.0,1.000000,大地电气,21.51
3,4,昨日连板,BK0816,10.10,0.000044,0.0,1.000000,大地电气,21.51
4,5,碳基材料,BK1006,3.05,0.000118,0.0,0.777778,翔丰华,19.99
5,6,数字哨兵,BK1084,3.00,0.000058,0.0,0.909091,汇纳科技,10.41
6,7,血氧仪,BK1122,2.84,0.000086,0.0,0.692308,中科美菱,29.87
7,8,AI语料,BK1172,2.64,0.000219,0.0,0.772727,福石控股,20.08
8,9,纳米银,BK0855,2.55,0.000074,0.0,0.833333,*ST有树,19.81
9,10,Web3.0,BK1110,2.34,0.000301,0.0,0.736842,福石控股,20.08


In [4]:
top5_concept_stocks=top5sectorStocks(basename,start_time,end_time,rank0)

排名前5领涨板块成分股列表。
板块：昨日涨停_含一字:
       代码    名称    涨跌幅  成交额  换手率  市盈率-动态    市净率
0  833030  立方控股  30.00  NaN  0.0  -51.93   5.91
1  834475  三友科技  29.95  NaN  0.0  211.52   6.84
2  870976  视声智能  28.80  NaN  0.0   53.13   9.35
3  870436  大地电气  21.27  NaN  0.0 -149.25  10.39
4  300071  福石控股  20.08  NaN  0.0 -137.36  21.41


板块：昨日涨停:
       代码    名称    涨跌幅  成交额  换手率  市盈率-动态    市净率
0  833030  立方控股  30.00  NaN  0.0  -51.93   5.91
1  834475  三友科技  29.95  NaN  0.0  211.52   6.84
2  870976  视声智能  28.80  NaN  0.0   53.13   9.35
3  870436  大地电气  21.27  NaN  0.0 -149.25  10.39
4  300071  福石控股  20.08  NaN  0.0 -137.36  21.41


板块：昨日连板_含一字:
       代码    名称    涨跌幅  成交额  换手率  市盈率-动态    市净率
0  870436  大地电气  21.27  NaN  0.0 -149.25  10.39
1  300840  酷特智能  20.00  NaN  0.0   23.84   3.73
2  603366  日出东方  10.04  NaN  0.0   96.39   2.16
3    2469  三维化学  10.01  NaN  0.0   29.31   2.18
4    2757  南兴股份  10.01  NaN  0.0   19.39   2.25


板块：昨日连板:
       代码    名称    涨跌幅  成交额  换手率   市盈率-动态    市净率
0  870436  大地电气  21.27

#### 2.2.3 开盘后15分钟概念板块排名

In [5]:
start_time = datetime.time(9, 31)
end_time = datetime.time(9, 52)
rank1=recordRank(basename,start_time,end_time)
rank1.head(20)

,排名,板块名称,板块代码,涨跌幅,市值比,换手率,上涨家数%,领涨股票,领涨股票-涨跌幅
0,1,昨日连板_含一字,BK1051,6.32,0.000066,4.47,0.800000,粤桂股份,9.97
1,2,昨日连板,BK0816,5.30,0.000042,3.92,0.800000,三维化学,10.01
2,3,低碳冶金,BK0979,2.99,0.000200,0.42,0.750000,河钢股份,7.83
3,4,破净股,BK1112,2.81,0.013240,0.33,0.912281,海宁皮城,9.93
4,5,光伏高速公路,BK1096,2.49,0.000067,0.52,0.857143,现代投资,9.90
5,6,滨海新区,BK0566,2.03,0.000146,0.98,0.681818,滨海能源,10.02
6,7,昨日涨停_含一字,BK1050,1.96,0.000370,5.73,0.628205,粤桂股份,9.97
7,8,昨日涨停,BK0815,1.49,0.000346,5.77,0.619718,粤桂股份,9.97
8,9,京津冀,BK0684,1.41,0.000397,1.04,0.629630,津投城开,10.05
9,10,有机硅,BK0961,1.21,0.000196,0.72,0.689655,硅宝科技,19.97


排名前5概念板块领涨成分股

In [6]:
top5_concept_stocks=top5sectorStocks(basename,start_time,end_time,rank1)

排名前5领涨板块成分股列表。
No record stored and not available from source.
No record stored and not available from source.
No record stored and not available from source.
No record stored and not available from source.
No record stored and not available from source.


#### 2.2.3 集合竞价行业板块涨跌幅

In [7]:
#当天集合竞价到开盘后30分钟涨跌幅排名
basename='industryRank'
start_time = datetime.time(9, 15)
end_time = datetime.time(9, 31)
rank3=recordRank(basename,start_time,end_time)
rank3.head(20)

非指定时间段，没有数据源.


""


行业板块中成分股

In [8]:
top5_industry_stocks=top5sectorStocks(basename,start_time,end_time,rank3)

非指定时间段，没有数据源。


#### 2.2.4 开盘15分钟行业板块涨跌幅

In [9]:
basename='industryRank'
start_time = datetime.time(9, 31)
end_time = datetime.time(9, 51)
rank4=recordRank(basename,start_time,end_time)
rank4.head(20)

,排名,板块名称,板块代码,涨跌幅,市值比,换手率,上涨家数%,领涨股票,领涨股票-涨跌幅
0,1,银行,BK0475,2.65,0.127240,0.14,1.000000,郑州银行,10.24
1,2,钢铁行业,BK0479,2.24,0.008715,0.52,0.857143,中南股份,10.15
2,3,房地产开发,BK0451,1.92,0.011952,0.88,0.815217,津投城开,10.05
3,4,煤炭行业,BK0437,1.89,0.019372,0.25,0.970588,上海能源,6.98
4,5,燃气,BK1028,1.64,0.002953,0.53,0.878788,水发燃气,10.00
5,6,电力行业,BK0428,1.43,0.030972,0.35,0.909091,郴电国际,10.05
6,7,航运港口,BK0450,1.13,0.010812,0.29,0.694444,天津港,5.57
7,8,医药商业,BK1042,1.03,0.003537,0.50,0.593750,嘉事堂,9.98
8,9,商业百货,BK0482,0.99,0.005044,0.81,0.568966,海宁皮城,9.93
9,10,保险,BK0474,0.97,0.031820,0.09,0.833333,天茂集团,1.79


行业板块中成分股

### 2.2.5 集合竞价阶段涨停个股热点明细

In [10]:
#在指定时段获取涨停数据，若时间不在指定时段，则读取历史最后一次下载的记录。
def stockLimitUp(start_time,end_time):
    now = datetime.datetime.now()
    tStr = end_time.strftime("%Y%m%d%H%M")
    csvfile=f'limitUp{tStr}.txt'
    if now.time() < end_time and now.time() >= start_time:
        rank=ak.stock_zh_a_spot_em()
        rank.to_csv(csvfile)
    else:
        if os.path.exists(csvfile):
            rank=pd.read_csv(csvfile)
        else:
            print('No record stored and not available from source.')
            rank=pd.DataFrame()
    if not rank.empty:
        columns=['代码','名称','涨跌幅','量比','换手率','市盈率-动态','市净率','流通市值','涨速','5分钟涨跌','60日涨跌幅','年初至今涨跌幅']
        rank=rank[columns].sort_values(by='涨跌幅', ascending=False)
    return rank

In [11]:
start_time = datetime.time(9, 15)
end_time = datetime.time(9, 31)
openA = stockLimitUp(start_time,end_time)
openA.head(50)

No record stored and not available from source.


""


### 2.2.6 开盘15分钟涨停个股热点明细


In [12]:
start_time = datetime.time(9, 31)
end_time = datetime.time(9, 53)
open15 = stockLimitUp(start_time,end_time)
open15.head(50)

,代码,名称,涨跌幅,量比,换手率,市盈率-动态,市净率,流通市值,涨速,5分钟涨跌,60日涨跌幅,年初至今涨跌幅
0,839719,宁新新材,29.97,13.89,14.34,-26.43,2.02,1.276002e+09,0.00,0.00,143.89,23.44
1,831689,克莱特,27.14,22.50,12.36,33.65,4.69,1.028783e+09,-2.20,5.46,170.69,87.43
2,834475,三友科技,25.88,35.96,22.65,204.90,6.63,1.047475e+09,0.96,-3.13,256.92,124.59
3,688173,希荻微,20.00,2.88,0.53,-22.29,3.68,3.379239e+09,0.00,0.00,50.16,-19.50
4,300019,硅宝科技,19.97,23.13,4.44,33.33,2.81,6.187320e+09,0.00,0.00,60.68,10.43
5,300840,酷特智能,17.91,45.00,38.69,23.42,3.66,3.472060e+09,-0.39,0.25,137.47,2.11
6,832175,东方碳素,16.76,10.73,14.33,-42.56,2.51,1.028386e+09,1.49,-2.13,108.85,37.54
7,832662,方盛股份,16.17,9.02,11.23,60.60,4.44,5.579816e+08,-2.58,-1.30,150.83,27.03
8,300981,中红医疗,15.50,28.26,2.48,85.40,1.03,5.325738e+09,-3.56,0.40,49.60,-3.99
9,688196,卓越新能,15.49,25.52,1.60,31.02,1.40,3.819600e+09,2.25,4.36,58.67,-4.41


#### 2.2.7 板块动态评析

本轮行情启动股票热点追踪（需要从历史行情中组合分析，现有股票涨停池昨日和今日都不稳定）

In [13]:
from datetime import datetime, timedelta

def get_stock_zt_pool(start_date, end_date):
    current_date = datetime.strptime(start_date, '%Y%m%d')
    end_date = datetime.strptime(end_date, '%Y%m%d')
    
    concat_zt_df = pd.DataFrame()

    while current_date <= end_date:
        date_str = current_date.strftime('%Y%m%d')
        try:
            stock_zt_pool_strong_em_df = ak.stock_zt_pool_previous_em(date=date_str)
            stock_zt_pool_strong_em_df['日期'] = pd.to_datetime(date_str)
            concat_zt_df = pd.concat([concat_zt_df, stock_zt_pool_strong_em_df], ignore_index=True)
        except Exception as e:
            print(f"Error fetching data for {date_str}: {e}")
        
        current_date += timedelta(days=1)
    
    return concat_zt_df

In [14]:
syclestart='20240925'
#ak.stock_zt_pool_strong_em(date=syclestart)

In [15]:
#本轮行情启动日
syclestart='20240924'
result_df = get_stock_zt_pool(syclestart, lastday)
result_df

NameError: name 'lastday' is not defined

## 4.个股市场


### 4.1 跟进股票列表
#### 4.1.1 历史K线

In [ ]:
def plotDcStockSingle(df,days):
    OCHLVDict={'日期':'Date','开盘':'Open','收盘':'Close','最高':'High','最低':'Low','成交量':'Volume'}
    print(f'{days}K线图')
    plotStock(df.tail(days),OCHLVDict)

In [ ]:
#根据代码获取股票信息
#symbol:若是字符串，则返回单个股信息；若是列表，则遍历元素，返回相应列表。
def getStockInfo(symbol):
    if type(symbol).__name__=='str':
        stocknamedf=ak.stock_individual_info_em(symbol=symbol)
        stockname=stocknamedf.loc[stocknamedf['item']=='股票简称','value'][1]
        stockIPO=stocknamedf.loc[stocknamedf['item']=='上市时间','value'][7]
        stockIndustry=stocknamedf.loc[stocknamedf['item']=='行业','value'][6]
        stockTAmount=stocknamedf.loc[stocknamedf['item']=='总股本','value'][2]
        stockFAmount=stocknamedf.loc[stocknamedf['item']=='流通股','value'][3]
        stockTValue=stocknamedf.loc[stocknamedf['item']=='总市值','value'][4]
        stockFValue=stocknamedf.loc[stocknamedf['item']=='流通市值','value'][5]
    else:
        stockname,stockIPO,stockIndustry,stockTAmount,stockFAmount,stockTValue,stockFValue=[],[],[],[],[],[],[]
        for sym in symbol:
            stocknamedf=ak.stock_individual_info_em(symbol=sym)
            stockname.append(stocknamedf.loc[stocknamedf['item']=='股票简称','value'][1])
            stockIPO.append(stocknamedf.loc[stocknamedf['item']=='上市时间','value'][7])
            stockIndustry.append(stocknamedf.loc[stocknamedf['item']=='行业','value'][6])
            stockTAmount.append(stocknamedf.loc[stocknamedf['item']=='总股本','value'][2])
            stockFAmount.append(stocknamedf.loc[stocknamedf['item']=='流通股','value'][3])
            stockTValue.append(stocknamedf.loc[stocknamedf['item']=='总市值','value'][4])
            stockFValue.append(stocknamedf.loc[stocknamedf['item']=='流通市值','value'][5])
            
    return {'名称':stockname,'IPO':stockIPO,'行业':stockIndustry,'总股本':stockTAmount,'流通股':stockFAmount,'总市值':stockTValue,'流通市值':stockFValue}

#画指定周期的股票烛线图
def plotDcStocks(stocksymb,periods):
    stockDict={}
    for p in periods:
        stockdf = ak.stock_zh_a_hist(symbol=stocksymb,period=p)
        stockname=getStockInfo(stocksymb)['名称']
        stockDict[p]=stockdf
        print(f'{stockname}-周期：{p}\n{stockdf}')
        plotDcStockSingle(stockdf,100)
    stockDict['name']=stockname
    return stockDict

In [ ]:
stocksymb='300339'
periods=['daily','weekly','monthly']
stockDict=plotDcStocks(stocksymb,periods)

### 4.1.2 盘口信息汇总
#### 4.1.2.1 成交和挂单明细

获取腾讯当日历史分笔数据

In [2]:
import akshare as ak

symbol='600839'
# 获取股票数据
df = ak.stock_bid_ask_em(symbol=symbol)

# 转换为字典
data_dict = df.set_index('item').to_dict()['value']

In [3]:
data_dict

{'sell_5': 14.24,
 'sell_5_vol': 54600.0,
 'sell_4': 14.23,
 'sell_4_vol': 42600.0,
 'sell_3': 14.22,
 'sell_3_vol': 176700.0,
 'sell_2': 14.21,
 'sell_2_vol': 169200.0,
 'sell_1': 14.2,
 'sell_1_vol': 1069500.0,
 'buy_1': 14.18,
 'buy_1_vol': 109800.0,
 'buy_2': 14.17,
 'buy_2_vol': 518400.0,
 'buy_3': 14.16,
 'buy_3_vol': 724400.0,
 'buy_4': 14.15,
 'buy_4_vol': 906200.0,
 'buy_5': 14.14,
 'buy_5_vol': 830600.0,
 '最新': 14.2,
 '均价': 14.03,
 '涨幅': 2.9,
 '涨跌': 0.4,
 '总手': 521208.0,
 '金额': 731469843.0,
 '换手': 1.13,
 '量比': 9.78,
 '最高': 14.2,
 '最低': 13.96,
 '今开': 13.96,
 '昨收': 13.8,
 '涨停': 15.18,
 '跌停': 12.42,
 '外盘': 249671.0,
 '内盘': 271537.0}

In [ ]:
# 根据code代码返回“交易所+代码” 信息，或者反过来
def transferExSym(code):
    if len(code)>6:
        rescode=code[-6:]
    else:
        source={}
        source['sh']=ak.stock_sh_a_spot_em()['代码'].to_list()
        source['sz']=ak.stock_sz_a_spot_em()['代码'].to_list()
        source['bj']=ak.stock_bj_a_spot_em()['代码'].to_list()
        for k in source.keys():
            if code in source[k]:
                rescode=k+code
                break
    return rescode

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#区分北交所和沪深股市，刻画市场深度。按照涨跌停板，作为极值，按照价格涨幅的measure度量区间
def priceMarketDepth(df,measure=0.01):

    open_price = df.loc[df['成交时间'] == '09:30:00', '成交价格'].iloc[0]

    # 设置涨跌停板系数
    if stocksymb == 'bj':
        limit_factor = 0.2
    else:
        limit_factor = 0.1

    # 计算最高价和最低价
    high_price = open_price * (1 + limit_factor)
    low_price = open_price * (1 - limit_factor)

    # 定义价格区间，每1%一档
    price_bins = np.round(np.arange(low_price, high_price, (high_price - low_price) * measure),2)

    # 将价格按照区间划分
    df['价格区间'] = pd.cut(df['成交价格'], bins=price_bins, right=False)

    # 按照价格区间汇总成交量
    volume_summary = df.groupby('价格区间',observed=False)['成交量'].sum().reset_index()

    # 新建 df['prop'] 列
    df['prop'] = df['性质'].apply(lambda x: -1 if x == '卖盘' else (1 if x == '买盘' else 0))
    
    # 按照价格区间汇总成交量和 prop 之和
    summary = df.groupby('价格区间',observed=False).agg({
        '成交量': 'sum',
        'prop': 'sum'
    }).reset_index()
    
    # 提取价格区间的中点和边界
    summary['价格'] = summary['价格区间'].apply(lambda x: round(x.mid,2))
    summary['起始价格'] = summary['价格区间'].apply(lambda x: round(x.left,2))
    summary['终止价格'] = summary['价格区间'].apply(lambda x: round(x.right,2))

    # 分离买盘和卖盘数据
    summary['买盘深度'] = summary['prop'].apply(lambda x: x if x > 0 else 0)
    summary['卖盘深度'] = -summary['prop'].apply(lambda x: -x if x < 0 else 0)

    # 绘制市场深度水平条形图
    fig, ax = plt.subplots(figsize=(14, 21))

    # 绘制买卖盘深度图
    ax.barh(summary['价格'], summary['买盘深度'], color='green', label='buy')
    ax.barh(summary['价格'], summary['卖盘深度'], color='red', label='sell')
    
    # 在图上标注区间价格的起始和终止价格
    for index, row in summary.iterrows():
        ax.text(row['买盘深度'] + max(summary['买盘深度'].max(), summary['卖盘深度'].max())*measure, row['价格'], f'{row["起始价格"]:.2f}', va='center', ha='left', fontsize=8, color='black')
        ax.text(-row['卖盘深度'] - max(summary['买盘深度'].max(), summary['卖盘深度'].max())*measure, row['价格'], f'{row["终止价格"]:.2f}', va='center', ha='right', fontsize=8, color='black')

    # 设置图表标题和标签
    ax.set_title('Market Depth')
    ax.set_xlabel('Depth(Amount)')
    ax.set_ylabel('Price')
    ax.legend()

    
    # 显示图表
    plt.show()
    
    return summary

In [ ]:
from rich.console import Console
from rich.table import Table

#美观显示DataFrame表格
class DataFramePretty(object):
    def __init__(self, df: pd.DataFrame, min_max_cols=[]) -> None:
        self.data = df
        self.min_max_cols = min_max_cols

    def __set_min_max_style(self, df: pd.DataFrame):
        imax = self.data.idxmax()
        imin = self.data.idxmin()

        max_tmpl = "[b on red3]{}[/b on red3]"
        min_tmpl = "[i on green3]{}[/i on green3]"

        for col in self.min_max_cols:
            col_idx = imax.index.tolist().index(col)
            val = df.iloc[imax[col], col_idx]
            df.iloc[imax[col], col_idx] = max_tmpl.format(val)

            col_idx = imin.index.tolist().index(col)
            val = df.iloc[imin[col], col_idx]
            df.iloc[imin[col], col_idx] = min_tmpl.format(val)

    def show(self):
        table = Table(
            title="DataFrame",
            title_style="i on dark_cyan",
            header_style="bold cyan",
        )

        # self.data是原始数据
        # df 是用来显示的数据
        df = self.data.copy()
        for col in df.columns:
            df[col] = df[col].astype("str")
            table.add_column(col)

        self.__set_min_max_style(df)

        for idx in range(len(df)):
            table.add_row(*df.iloc[idx].tolist())

        console = Console()
        console.print(table)

In [ ]:
#剔除中性单以后，列出活跃买卖市场深度
def plotActiveMarketDepth(stocksymbol):
    
    print(f'市场深度：{stocksymb}')
    #一次性获得历史分笔数据（当天即当天之前的交易时间；隔天即昨天）
    yesterdayTicks=ak.stock_zh_a_tick_tx_js(symbol=transferExSym(stocksymb))
    
    md=priceMarketDepth(yesterdayTicks)
    md = md[(md['买盘深度'] != 0) | (md['卖盘深度'] != 0)]
    mdp=DataFramePretty(md)
    mdp.show()
    return  md

剔除中性盘以后，主动买卖盘的市场深度

In [ ]:
plotActiveMarketDepth(stocksymb)

### 4.1.3 个股追踪
追踪股票列表

In [ ]:
focusStocks=['300576','600839','600157','000717','002607','600127','600605','600217','300727','301421','002843','600968','300159','601688']
getInfo=getStockInfo(focusStocks)
Analist=pd.DataFrame({'代码':focusStocks,'名称':getInfo['名称'],'IPO':getInfo['IPO'],'行业':getInfo['行业'],'流通股':getInfo['流通股'],'总股本':getInfo['总股本']})
Analist['流通股']=Analist['流通股'].astype(int)
Analist['总股本']=Analist['总股本'].astype(int)
Analist['流通率']=Analist['流通股']/Analist['总股本']
Analist

时序数据库准备

In [ ]:
import dolphindb as ddb
from concurrent.futures import ThreadPoolExecutor
import dolphindb.settings as keys

s = ddb.session("localhost", 8848, "admin", "123456")
s.run("quote_tab = table(1:0, `timestamp`sym`sell_5`sell_5_vol`sell_4`sell_4_vol`sell_3`sell_3_vol`sell_2`sell_2_vol`sell_1`sell_1_vol`buy_1`buy_1_vol`buy_2`buy_2_vol`buy_3`buy_3_vol`buy_4`buy_4_vol`buy_5`buy_5_vol`最新`均价`涨幅`涨跌`总手`金额`换手`量比`最高`最低`今开`昨收`涨停`跌停`外盘`内盘, [TIMESTAMP, SYMBOL, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE,DOUBLE,DOUBLE])")
quote_tab = s.table(data="quote_tab")
dbPath="dfs://quotes"

if not s.existsDatabase(dbPath):
    db = s.database(dbPath=dbPath, partitionType=keys.LIST, partitions=focusStocks,engine='TSDB')
    db.createTable(quote_tab, "quotes", ["sym"])
    
qt = s.table(dbPath=dbPath, data="quotes")


In [ ]:
stockCharts,marketDepths={},{}

for f in focusStocks:
    print('='*75)
    stockCharts[f]=plotDcStocks(f,periods)
    stockname=stockCharts[f]['name']
    print(f'{stockname}（{f}）的市场深度：')
    marketDepths[f]=plotActiveMarketDepth(f)
    

In [ ]:
stock_bid_ask_em_df = ak.stock_bid_ask_em(symbol="600839")
stock_bid_ask_em_df

In [ ]:
import akshare as ak
import dolphindb as ddb
import pandas as pd
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

# 初始化 DolphinDB 连接
s = ddb.session()
s.connect("localhost", 8848, "admin", "123456")  # 替换为实际的主机、端口、用户名和密码

# 创建 DolphinDB 表
s.run("""
if (existsDatabase("dfs://Quote")) {
    dropDatabase("dfs://Quote")
}
t = table(1:0, `timestamp`sym`sell_5`sell_5_vol`sell_4`sell_4_vol`sell_3`sell_3_vol`sell_2`sell_2_vol`sell_1`sell_1_vol`buy_1`buy_1_vol`buy_2`buy_2_vol`buy_3`buy_3_vol`buy_4`buy_4_vol`buy_5`buy_5_vol`最新`均价`涨幅`涨跌`总手`金额`换手`量比`最高`最低`今开`昨收`涨停`跌停`外盘`内盘, [TIMESTAMP, SYMBOL, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE])
database("dfs://Quote", VALUE, t)
""")

def is_within_trading_hours(current_time):
    # 定义交易时间段
    morning_start = current_time.replace(hour=9, minute=15, second=0, microsecond=0)
    morning_end = current_time.replace(hour=11, minute=30, second=0, microsecond=0)
    afternoon_start = current_time.replace(hour=13, minute=0, second=0, microsecond=0)
    afternoon_end = current_time.replace(hour=15, minute=0, microsecond=0)
    
    return (morning_start <= current_time <= morning_end) or (afternoon_start <= current_time <= afternoon_end)

def fetch_and_store(symbol):
    # 获取当前时间
    current_time = datetime.now()

    if is_within_trading_hours(current_time):fgggttryy
        # 获取股票数据
        df = ak.stock_bid_ask_em(symbol=symbol)

        # 转换为字典
        data_dict = df.set_index('item').to_dict()['value']

        # 添加时间戳和股票代码到数据字典
        data_dict['timestamp'] = current_time
        data_dict['sym'] = symbol

        # 转换数据为 DataFrame
        transformed_df = pd.DataFrame([data_dict])

        # 将数据插入到 DolphinDB
        s.upload({"data": transformed_df})
        s.run("""
        append!(loadTable("dfs://Quote", "t"), data)
        """)]]
        ]

while True:
    
    # 使用多线程获取多个股票代码的数据
    with ThreadPoolExecutor(max_workers=len(focusStocks)) as executor:
        executor.map(fetch_and_store, focusStocks)

    # 每3秒检查一次
    time.sleep(3)
